# Model integration

This tutorial demonstrate how an openalea model can be exported as an IPM-compliant web service

## Python Model creation

Let export the folowing simple model, that return some risk level, as a function of air temperature and a user-defined temperature threshold:

In [21]:
def t_risk(tair, threshold=15):
    if tair <= threshold:
        return 0
    else:
        return 1

In [22]:
t_risk(0), t_risk(20), t_risk(20, threshold=20)

(0, 1, 0)

## Import in OpenAlea

Let import it in OpenAlea, by embending it in a OpenAlea Node:

In [23]:
from openalea.core.node import FuncNode
from openalea.core import IFloat, IInt

In [24]:
inputs = (dict(name='tair', interface=IFloat, value=None),
          dict(name='threshold', interface=IFloat, value=15))
outputs = (dict(name='Risk', interface=IInt), )
my_node = FuncNode(inputs, outputs, t_risk)
my_node.name='TRISK'

In [25]:
my_node((0,)), my_node((20,)),my_node((20,20))

(0, 1, 0)

In [26]:
my_node.name, my_node.input_desc, my_node.output_desc

('TRISK',
 [{'name': 'tair', 'interface': IFloat, 'value': None},
  {'name': 'threshold', 'interface': IFloat, 'value': 15}],
 [{'name': 'Risk', 'interface': IInt}])

## Create IPM model

In [27]:
from openalea.dss import Manager
from openalea.dss.dss_factory import dss_factory
help(dss_factory)

Help on function dss_factory in module openalea.dss.dss_factory:

dss_factory(node, interval=86400, weather_parameters=None, parameters=None, decision_support=None, template=None)
    Transform an openalea node in a IPM model json descriptor and generate a fastAPI script to launch webservice
    
    Args:
        node: the node to be exported
        interval: the time step of the model (s)
        weather_parameters: a mapping between node input name and weather data codes, if any.
        None if none of the input is a weather data
        parameters: a list of node input name to be exposed as config parameters in IPM-Decison platform
        template: (optional) an existing IPM model to be used as a template for filling missing information
    Returns:
        model: a json-like dict describing the model
        dss_service: a string containing the script to be run for launching the web service



We will use as a templase a nibio vips model, and define a simple decision_support function that help a user interpret the risk level of our model

In [28]:
decision_support = list(range(2))
decision_support[0] = {'explanation': 'Risk is low',
                      'recommended_action': 'No particular action is required'}
decision_support[1] = {'explanation': 'Risk is high',
                      'recommended_action': 'Be carreful !'}

In [29]:
m = Manager()
template = m.get_model("no.nibio.vips","PSILARTEMP")

In [30]:
ipm_model, service = dss_factory(my_node, weather_parameters={'tair': 1002},parameters=['threshold'],decision_support=decision_support,template=template)

In [31]:
ipm_model

{'name': 'Carrot fly flight model',
 'id': 'TRISK',
 'version': '1.0',
 'purpose': 'Estimates risk of flight and egg laying in crop',
 'description': 'THE PEST: The first generation of adult carrot fly emerge from pupae in the soil in the spring, and lay eggs close to the base of vulnerable crops. Larvae initial feed at the surface, then tunnel into the tap root. Adults emerge mid-July and can lead to a second generation. \nTHE DECISION: Treatments may need to be applied soon after adults arrive in the crop, before larvae tunnel into the crop roots.  \nTHE MODEL: The model determines the start of the flight period for the 1st generation of carrot rust fly based on accumuleted degree-days (260 day-degrees) over a base temperature of 5°C.  \nTHE PARAMETERS: The model uses daily air temperature \nSOURCE: Luke, Finland. \nASSUMPTIONS: Be aware that in areas with field covers (plastic, single or double non-woven covers, etc.) with early crops the preceding season (either on the current fiel

## Launch the Webservice

In [39]:
from openalea.dss.dss_factory import start_service, stop_service

http, handler = start_service(service)

In [40]:
from IPython.display import IFrame

In [41]:
IFrame(http + '/docs', 800,400)

## Test the exported model

In [42]:
from agroservices.ipm.ipm import IPM
import agroservices.ipm.fakers as ipm_fakers
import json

In [43]:
ipm_model['execution']['input_schema']

{'type': 'object',
 'properties': {'modelId': {'type': 'string',
   'pattern': '^TRISK$',
   'title': 'Model Id',
   'default': 'TRISK',
   'description': 'Must be TRISK'},
  'configParameters': {'title': 'Configuration parameters',
   'type': 'object',
   'properties': {'timeStart': {'type': 'string',
     'format': 'date',
     'default': '{CURRENT_YEAR}-03-01',
     'title': 'Start date of calculation (YYYY-MM-DD)'},
    'timeEnd': {'type': 'string',
     'format': 'date',
     'default': '{CURRENT_YEAR}-09-01',
     'title': 'End date of calculation (YYYY-MM-DD)'},
    'threshold': {'default': 15, 'type': 'number'}},
   'required': ['timeStart', 'timeEnd', 'threshold']},
  'weatherData': {'$ref': 'https://platform.ipmdecisions.net/api/wx/rest/schema/weatherdata'}},
 'required': ['modelId', 'configParameters']}

In [44]:
ipm = IPM()
input_data = ipm_fakers.input_data(ipm_model)
input_data

{'modelId': 'TRISK',
 'configParameters': {'timeStart': '2023-03-01',
  'timeEnd': '2023-09-01',
  'threshold': 15.0},
 'weatherData': {'weatherParameters': [1002],
  'timeStart': '2023-03-01T00:00:00+01:00',
  'timeEnd': '2023-09-01T00:00:00+02:00',
  'interval': 86400,
  'locationWeatherData': [{'longitude': '16.54165',
    'latitude': '68.79833',
    'altitude': 0,
    'data': [[6.9],
     [6.6],
     [1.8],
     [9.4],
     [9.6],
     [3.8],
     [5.6],
     [5.0],
     [9.8],
     [9.2],
     [1.2],
     [0.9],
     [6.4],
     [9.6],
     [0.5],
     [3.9],
     [4.1],
     [1.3],
     [1.1],
     [1.3],
     [4.2],
     [7.7],
     [7.7],
     [9.9],
     [6.8],
     [3.8],
     [2.1],
     [5.2],
     [9.0],
     [0.9],
     [2.8],
     [0.6],
     [2.1],
     [9.1],
     [1.9],
     [6.7],
     [0.0],
     [1.3],
     [5.9],
     [7.3],
     [1.7],
     [3.1],
     [1.9],
     [7.8],
     [2.0],
     [9.5],
     [2.6],
     [4.6],
     [7.6],
     [9.9],
     [1.9],
     [8.3

In [45]:
ipm.run_model(ipm_model, input_data)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# Stop service

In [46]:
stop_service(handler)

In [38]:
IFrame(http + '/docs', 800,400)